In [8]:
import cv2
import numpy as np

def motion_blur_psf(length=15, angle=0):
    psf = np.zeros((length, length))
    # 在中心畫一條線
    psf[length // 2, :] = 1
    # 旋轉以改變模糊方向
    M = cv2.getRotationMatrix2D((length / 2, length / 2), angle, 1)
    psf = cv2.warpAffine(psf, M, (length, length))
    psf /= psf.sum()  # 正規化
    return psf

def fft_convolve(img, psf):
    # 如果是彩色圖，分通道處理
    channels = cv2.split(np.float32(img))
    blurred_channels = []

    for c in channels:
        psf_pad = np.zeros_like(c)
        kh, kw = psf.shape
        psf_pad[:kh, :kw] = psf
        psf_pad = np.roll(psf_pad, -kh//2, axis=0)
        psf_pad = np.roll(psf_pad, -kw//2, axis=1)

        # 頻域卷積
        img_f = cv2.dft(c, flags=cv2.DFT_COMPLEX_OUTPUT)
        psf_f = cv2.dft(psf_pad, flags=cv2.DFT_COMPLEX_OUTPUT)
        blurred_f = cv2.mulSpectrums(img_f, psf_f, 0)
        blurred = cv2.idft(blurred_f, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
        blurred_channels.append(blurred)

    # 合併通道
    blurred_img = cv2.merge(blurred_channels)
    return np.clip(blurred_img, 0, 255).astype(np.uint8)

img = cv2.imread("original/cat.jpg")
psf = motion_blur_psf(length=50, angle=30)
blurred = fft_convolve(img, psf)
cv2.imwrite("input/cat.png", blurred)


True